In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import time
import hashlib
import os
import tempfile

In [11]:
df = pd.read_csv(
    "/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/CT_metdata_copy_1.csv"
)
paper_url = df["doi"].to_list()

In [12]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'cord_uid', 'sha', 'source_x', 'title',
       'doi', 'pmcid', 'pubmed_id', 'license', 'abstract', 'publish_time',
       'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id',
       'pdf_json_files', 'pmc_json_files', 'url', 's2_id', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'doi_replace', 'abstract_replace', 'crossref_title',
       'crossref_abstract', 'crossref_journal', 'crossref_authors',
       'crossref_publish_time', 'issn', 'crossref_publication_type'],
      dtype='object')

In [10]:
df["doi"].unique().tolist()

['10.1186/cc987',
 '10.1186/cc1013',
 '10.1093/ecam/nel003',
 '10.1186/1465-9921-7-74',
 '10.1186/1471-2458-6-126',
 '10.1186/1476-0711-5-7',
 '10.1186/cc4981',
 '10.1371/journal.pone.0000143',
 '10.1186/cc5944',
 '10.1186/cc5675',
 '10.1093/nar/gkn518',
 '10.1371/journal.pone.0003548',
 '10.1186/1750-1172-3-34',
 '10.1371/journal.pone.0006314',
 '10.1186/cc7875',
 '10.1007/s00281-009-0164-5',
 '10.2174/138161208786898806',
 '10.1186/cc8183',
 '10.1051/vetres/2009071',
 '10.1371/journal.pone.0009332',
 '10.1155/2010/174378',
 '10.3201/eid1310.061262',
 '10.1093/ecam/nep152',
 '10.1371/journal.pone.0011721',
 '10.1203/pdr.0b013e3181e9f3a0',
 '10.1371/journal.pone.0012262',
 '10.1186/1750-1172-5-22',
 '10.1371/journal.pone.0013137',
 '10.1371/journal.pmed.1000346',
 '10.1371/journal.pone.0013047',
 '10.1371/journal.pmed.1000362',
 '10.1007/s00705-010-0768-z',
 '10.1371/journal.pntd.0000892',
 '10.1093/nar/gkq650',
 '10.1371/journal.pone.0015301',
 '10.1155/2010/817580',
 '10.1186/2042-64

In [4]:
retrieved = []

In [13]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if row['doi'] not in retrieved:
        url = "https://api.crossref.org/works/{}".format(row["doi"])
        req = requests.get(url)
        
        if req.status_code == 200:
            data = req.json()
            df.at[index, "crossref_title"] = data["message"].get("title", "NA")
            df.at[index, "crossref_abstract"] = data["message"].get("abstract", "NA")
            df.at[index, "crossref_journal"] = data["message"].get("container-title", "NA")
            df.at[index, "crossref_authors"] = data["message"].get("author", "NA")
            df.at[index, "crossref_publish_time"] = data["message"]["created"].get("date-time", "NA")
            df.at[index, "issn"] = data["message"].get("ISSN", "NA")
            df.at[index, "crossref_publication_type"] = data["message"].get("type", "NA")
            retrieved.append(row["doi"])
            
        else:
            df.at[index, "crossref_title"] = "NA"
            df.at[index, "crossref_abstract"] = "NA"
            df.at[index, "crossref_journal"] = "NA"
            df.at[index, "crossref_authors"] = "NA"
            df.at[index, "crossref_publish_time"] = "NA"
            df.at[index, "issn"] = "NA"
            df.at[index, "crossref_publication_type"] = "NA"
            retrieved.append(row["doi"])
        
        df.to_csv("/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/CT_metdata_copy_1.csv",index=False)
        # Sleep for a short period to avoid hitting the rate limit
        time.sleep(0.9)
        

100%|██████████| 8722/8722 [27:04<00:00,  5.37it/s]   


In [125]:
nan_mask = df[['crossref_abstract','issn']].isna().all(axis=1)
df[nan_mask]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,Unnamed: 31,doi_replace,abstract_replace,crossref_title,crossref_abstract,crossref_journal,crossref_authors,crossref_publish_time,issn,crossref_publication_type
2316,4q5edir8,b18e1ab81f9263fe8f2ec5bec5c54c31371f4a4c,PMC,Delivery Systems for In Vivo Use of Nucleic Ac...,NaN,PMC3155220,21901073,no-cc,The notorious biotechnological advance of the ...,2007-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2317,oez3l9kn,f3042820ac417ec01ca0e15a4232e24b8d78ba74,PMC,Targeting Human Immunodeficiency Virus Type 1 ...,NaN,PMC3155237,21901072,no-cc,The targets for licensed drugs used for the tr...,2007-07-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2318,ra756rn1,NaN,PMC,Pre-clinical toxicity & immunobiological evalu...,NaN,PMC3734712,23852288,cc-by-nc-sa,BACKGROUND & OBJECTIVES: Pre-clinical toxicolo...,2013-06-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2319,4jycukj3,8e014e2133b319d93ba7e9341ffe33bab3f9e265,PMC,A HLA-A2-restricted CTL epitope induces anti-t...,NaN,PMC4808025,26621839,cc-by,Cancer immunotherapy is attractive for antigen...,2015-11-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2321,l0asrcn5,655b175741b74ae8c98f93fd449c88a147bee96e,PMC,Prevalence of Underlying Diseases in Hospitali...,NaN,PMC7096724,NaN,cc-by,"INTRODUCTION: In the beginning of 2020, an une...",2020-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2324,koaaxltk,3afd5d184312c4501ab9038756e7297e3e87b1d5,PMC,COVID-19 and Ophthalmology: A New Chapter in a...,NaN,PMC7134243,NaN,cc-by,"Reviglio, Victor Eduardo; Osaba, Matias; Revig...",2020-03-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2325,2qqkoeo9,a4a07b11d77ec96452acb8806b274d7654a72ed4,PMC,COVID-19: Preliminary Clinical Guidelines for ...,NaN,PMC7141793,NaN,cc-by,The zoonotic Severe Acute Respiratory Syndrome...,2020-04-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2326,jlqee1b8,a9fd69bd68f51ed7f40652d85681cb66a9d66ba7,PMC,Antiviral therapy in management of COVID-19: a...,NaN,PMC7156260,NaN,cc-by,BACKGROUND: The purpose of the current systema...,2020-04-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2327,qz8pffei,27521ddbfb535760a7e4d77ac0cd6a0f3ebfb36a,PMC,COVID-19 in Singapore and Malaysia: Rising to ...,NaN,PMC7169474,NaN,cc-by,With the increasing number of COVID-19 cases a...,2020-04-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2328,qvcbfnbf,15af325f092c29298a36e80431a7837ea6961439,PMC,Review of safety and minimum pricing of nitazo...,NaN,PMC7213070,NaN,cc-by-nc,BACKGROUND: Many treatments are being assessed...,2020-04-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
len(retrieved)

8708

In [17]:
retrieved

['10.1186/cc987',
 '10.1186/cc1013',
 '10.1093/ecam/nel003',
 '10.1186/1465-9921-7-74',
 '10.1186/1471-2458-6-126',
 '10.1186/1476-0711-5-7',
 '10.1186/cc4981',
 '10.1371/journal.pone.0000143',
 '10.1186/cc5944',
 '10.1186/cc5675',
 '10.1093/nar/gkn518',
 '10.1371/journal.pone.0003548',
 '10.1186/1750-1172-3-34',
 '10.1371/journal.pone.0006314',
 '10.1186/cc7875',
 '10.1007/s00281-009-0164-5',
 '10.2174/138161208786898806',
 '10.1186/cc8183',
 '10.1051/vetres/2009071',
 '10.1371/journal.pone.0009332',
 '10.1155/2010/174378',
 '10.3201/eid1310.061262',
 '10.1093/ecam/nep152',
 '10.1371/journal.pone.0011721',
 '10.1203/pdr.0b013e3181e9f3a0',
 '10.1371/journal.pone.0012262',
 '10.1186/1750-1172-5-22',
 '10.1371/journal.pone.0013137',
 '10.1371/journal.pmed.1000346',
 '10.1371/journal.pone.0013047',
 '10.1371/journal.pmed.1000362',
 '10.1007/s00705-010-0768-z',
 '10.1371/journal.pntd.0000892',
 '10.1093/nar/gkq650',
 '10.1371/journal.pone.0015301',
 '10.1155/2010/817580',
 '10.1186/2042-64

In [3]:
url = "https://api.crossref.org/works/10.1016/j.resp.2020.103476"
r = requests.get(url)
if r.status_code == 200:
    data = r.json()
    issn = data["message"]["ISSN"][0] if "ISSN" in data["message"] else None

In [6]:
data['message']

{'indexed': {'date-parts': [[2023, 3, 20]],
  'date-time': '2023-03-20T10:50:33Z',
  'timestamp': 1679309433739},
 'reference-count': 113,
 'publisher': 'Elsevier BV',
 'license': [{'start': {'date-parts': [[2020, 8, 1]],
    'date-time': '2020-08-01T00:00:00Z',
    'timestamp': 1596240000000},
   'content-version': 'tdm',
   'delay-in-days': 0,
   'URL': 'https://www.elsevier.com/tdm/userlicense/1.0/'}],
 'funder': [{'name': '“Fonds de recherche du Quebec-Santé”',
   'award': ['FRQ-S; FQ121919']},
  {'name': 'IUCPQ Foundation'},
  {'DOI': '10.13039/501100000925',
   'name': 'NHMRC',
   'doi-asserted-by': 'publisher',
   'award': ['APP1165529']}],
 'content-domain': {'domain': ['clinicalkey.jp',
   'clinicalkey.com',
   'clinicalkey.es',
   'clinicalkey.fr',
   'clinicalkey.com.au',
   'elsevier.com',
   'sciencedirect.com'],
  'crossmark-restriction': True},
 'short-container-title': ['Respiratory Physiology &amp; Neurobiology'],
 'published-print': {'date-parts': [[2020, 8]]},
 'DOI'

In [128]:
data["message"].keys()

dict_keys(['indexed', 'reference-count', 'publisher', 'issue', 'license', 'content-domain', 'short-container-title', 'published-print', 'DOI', 'type', 'created', 'page', 'update-policy', 'source', 'is-referenced-by-count', 'title', 'prefix', 'volume', 'author', 'member', 'reference', 'container-title', 'original-title', 'language', 'link', 'deposited', 'score', 'resource', 'subtitle', 'short-title', 'issued', 'references-count', 'journal-issue', 'alternative-id', 'URL', 'relation', 'ISSN', 'issn-type', 'subject', 'published', 'assertion'])

In [132]:
data["message"]['title']

['Thoracic Anesthesia in the Coronavirus Disease 2019 (COVID-19) Era']